In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
detector = tf.keras.models.load_model("efficientdet_lite2_detection_1")
model = tf.keras.models.load_model('saved_model/my_model')

In [2]:
def rectangle_to_square(coords):
    # get the coordinates of the rectangle
    ymin, xmin, ymax, xmax = coords
    
    # calculate the width and height of the rectangle
    width = xmax - xmin
    height = ymax - ymin
    
    # calculate the center of the rectangle
    cx = xmin + (width / 2)
    cy = ymin + (height / 2)
    
    # determine the length of the square
    length = max(width, height)
    
    # calculate the new coordinates of the square
    square_coords = [
        int(cy - (length / 2)),
        int(cx - (length / 2)),
        int(cy + (length / 2)),
        int(cx + (length / 2))
    ]
    
    return square_coords

In [14]:
# cap = cv2.VideoCapture(1)

# #Capture frame-by-frame
# ret, frame = cap.read()

# frame = cv2.imread('common_photos/envelopes/IMG_3629.jpg')
frame = cv2.imread('common_photos/test/PXL_20230427_171056248.jpg')

width = 512
height = 512

#Resize to respect the input_shape
inp = cv2.resize(frame, (width , height ))

#Convert img to RGB
rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

#Is optional but i recommend (float convertion and convert img to tensor image)
rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

#Add dims to rgb_tensor
rgb_tensor = tf.expand_dims(rgb_tensor , 0)

boxes, scores, classes, num_detections = detector(rgb_tensor)

pred_boxes = boxes.numpy()[0].astype('int')
pred_scores = scores.numpy()[0]

#loop throughout the detections and place a box around it  
for score, (ymin,xmin,ymax,xmax) in zip(pred_scores, pred_boxes):
    if score < 0.2:
        continue

    # Transform rectangle into square to fit into NN
    ymin,xmin,ymax,xmax = rectangle_to_square((ymin,xmin,ymax,xmax))

    # Make sure ROI is within bounds of image
    ymin = max(0, ymin)
    ymax = min(512, ymax)
    xmin = max(0, xmin)
    xmax = min(512, xmax)

    # Isolate part of image with object
    object_region = frame[ymin:ymax, xmin:xmax]

    # Reshape region to fit NN
    object_region = cv2.resize(object_region, (160 , 160 ))

    # convert image to RGB
    object_tensor = cv2.cvtColor(object_region, cv2.COLOR_BGR2RGB)

    # Transform region into tensor form
    object_tensor = tf.convert_to_tensor(object_region, dtype=tf.uint32)

    # Add dims to rgb_tensor
    object_tensor = tf.expand_dims(object_tensor , 0)

    # Use the pre-trained model to make a prediction
    prediction = model.predict(object_tensor)[0]

    # Get the predicted label
    predictions = tf.nn.sigmoid(prediction)

    label = ("Compostable" if predictions < 0.01 else "Non-Compostable")
    confidence_score = (1-predictions if predictions < 0.01 else predictions)

    label_text = f'{label} => {confidence_score[0]*100:.8f}% Confidence'
        
    img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),1)      
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img_boxes,label_text,(xmin, ymax-10), font, 0.5, (0,0,255), 1, cv2.LINE_AA)



#Display the resulting frame
cv2.imshow('black and white',img_boxes)

# When everything done, release the capture
cv2.waitKey(0)
# cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 110ms/step
